In [1]:
import cv2
import os
import time
import numpy as np
from keras.models import load_model
from pygame import mixer

Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()
alarm_sound = mixer.Sound('alarm.wav')

In [3]:
haar_cascade_face = cv2.CascadeClassifier('haar_cascade_files\haarcascade_frontalface_alt.xml')
haar_cascade_left_eye = cv2.CascadeClassifier('haar_cascade_files\haarcascade_lefteye_2splits.xml')
haar_cascade_right_eye = cv2.CascadeClassifier('haar_cascade_files\haarcascade_righteye_2splits.xml')

In [9]:
model = load_model('model/cnn_model.h5')

In [10]:
path_cwd = os.getcwd()
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
thickness = 2
score = 0
left_eye_prediction = [99]
right_eye_prediction = [99]

In [18]:
l = time.time()
print(l)

1584359698.3206842


In [12]:
image_video = cv2.VideoCapture(0)
while(True):
    ret, frame = image_video.read()
    height, width = frame.shape[:2] 

    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = haar_cascade_face.detectMultiScale(gray_image, minNeighbors=5, scaleFactor=1.1, minSize=(25,25))
    left_eye = haar_cascade_left_eye.detectMultiScale(gray_image)
    right_eye = haar_cascade_right_eye.detectMultiScale(gray_image)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        right_eye_value = frame[y:y+h,x:x+w]        
        right_eye_value = cv2.cvtColor(right_eye_value, cv2.COLOR_BGR2GRAY)
        right_eye_value = cv2.resize(right_eye_value,(24,24))
        right_eye_value = right_eye_value/255
        right_eye_value = right_eye_value.reshape(24,24,-1)
        right_eye_value = np.expand_dims(right_eye_value,axis=0)
        right_eye_prediction = model.predict_classes(right_eye_value)
        break

    for (x,y,w,h) in left_eye:
        left_eye_value = frame[y:y+h,x:x+w]        
        left_eye_value = cv2.cvtColor(left_eye_value, cv2.COLOR_BGR2GRAY)  
        left_eye_value = cv2.resize(left_eye_value, (24,24))
        left_eye_value = left_eye_value/255
        left_eye_value = left_eye_value.reshape(24,24,-1)
        left_eye_value = np.expand_dims(left_eye_value,axis=0)
        left_eye_prediction = model.predict_classes(left_eye_value)
        break
    
    
    if(right_eye_prediction[0] == 0 and left_eye_prediction[0] == 0):
        score = score + 1
        cv2.putText(frame, "Eye Closed", (10, height - 20), font, 1, (255,255,255), 1, cv2.LINE_AA)    
    else:
        score = score - 1
        cv2.putText(frame, "Eye Opened", (10, height - 20), font, 1,(255,255,255), 1, cv2.LINE_AA)
         
    if(score < 0):
        score = 0
        alarm_sound.stop()
        
    cv2.putText(frame, 'Score:'+str(score), (100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    if(score > 15):
        #person is feeling sleepy so we beep the alarm
        #cv2.imwrite(os.path.join(path_cwd,'image.jpg'),frame)
        try:
            alarm_sound.play()            
        except:
            pass
        if(thickness < 16):
            thickness= thickness +2
        else:
            thickness = thickness - 2
            if(thickness < 2):
                thickness = 2
        cv2.rectangle(frame, (0,0), (width,height), (0,0,255), thickness) 
    cv2.imshow('Driver Drowsiness Detection',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
image_video.release()
cv2.destroyAllWindows()

12
11
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
